In [15]:
import cv2
import math
import numpy as np

In [16]:
def add_line_shadows(image, shadow_intensity=50, dash_width=10, dash_spacing=20, offset=0, rotation=0):
    h, w, c = image.shape

    # Diagonal length
    d = math.ceil((h**2 + w**2)**0.5)

    # Mask is larger to accomodate the rotation
    mask = np.zeros((d, d, c), dtype=np.uint8)

    # Number of dashs on mask
    dash_count = d // (dash_width + dash_spacing)

    # Create dashes on the mask
    for i in range(dash_count):
        x = i * (dash_width + dash_spacing) + dash_spacing
        cv2.rectangle(mask, (x, 0), (x + dash_width, d), (shadow_intensity, shadow_intensity, shadow_intensity), -1)

    # Calculate rotation matrix using the center of the mask
    mask_center = (d // 2, d // 2)
    rotation_matrix = cv2.getRotationMatrix2D(mask_center, rotation, 1)

    # Rotate the mask
    rotated_mask = cv2.warpAffine(mask, rotation_matrix, (d, d))

    # Crop mask
    x_crop = (d - w) // 2
    y_crop = (d - h) // 2
    cropped_mask = rotated_mask[y_crop:y_crop + h, x_crop:x_crop + w]

    # Apply cropped mask
    shadowed_image = cv2.subtract(image, cropped_mask)

    return shadowed_image

In [17]:
image = cv2.imread('main_figure.png')
print(image.shape)
cv2.imshow('shadowed img',add_line_shadows(image, rotation=30))
cv2.waitKey(0)
cv2.destroyAllWindows()

(619, 2044, 3)


QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread (0x19bbff0).
Cannot move to target thread (0x1851370)

QObject::moveToThread: Current thread (0x1851370) is not the object's thread